In [1]:
# Not using GPU
# Stop using CUDA
# import os

# # Set this BEFORE importing stumpy
# os.environ["NUMBA_DISABLE_CUDA"] = "1"

In [2]:
# Using GPU
import os
# Must be set before importing libraries that use the GPU!
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID" 
os.environ["CUDA_VISIBLE_DEVICES"] = "6" # Example: Only expose the RTX 5880 (Index 6)

In [3]:
import pandas as pd
import numpy as np
import random
random.seed(42)
from datetime import datetime, timedelta
from sklearn import preprocessing
%load_ext autoreload
%autoreload 2
from GBRT_for_TSF.utils import evaluate_with_xgboost
from mpmf.utils import get_top_1_motif_numba

In [4]:
def New_preprocessing(
    TimeSeries,
    num_periods_input,
    num_periods_output,
    include_itself,
    include_covariates,
    include_motif_information,
    k_motifs,
    no_points_after_motif,
    do_normalization=False,
    include_similarity=False
):
    Data = []
    # Change 1
    #################################################################################################
    start_date = datetime(2012, 5, 1, 00, 00, 00)  # define start date
    for i in range(0, len(TimeSeries)):
        record = []
        record.append(TimeSeries[i])  # adding the pemds7 value
        record.append(start_date.month)
        record.append(start_date.day)
        record.append(start_date.hour)
        record.append(start_date.minute)
        record.append(start_date.weekday())
        record.append(start_date.timetuple().tm_yday)
        record.append(start_date.isocalendar()[1])
        start_date = start_date + timedelta(minutes=5)
        Data.append(record)
    headers = [
        "pems",
        "month",
        "day",
        "hour",
        "minute",
        "day_of_week",
        "day_of_year",
        "week_of_year",
    ]
    Data_df = pd.DataFrame(Data, columns=headers)
    # print("Shape of Data_df:", Data_df.shape)
    sub = Data_df.iloc[:, 1:]
    # Normalize features to be from -0.5 to 0.5 as mentioned in the paper
    New_sub = preprocessing.minmax_scale(sub, feature_range=(-0.5, 0.5))
    # print("Shape of New_sub:", New_sub.shape)
    Normalized_Data_df = pd.DataFrame(
        np.column_stack([Data_df.iloc[:, 0], New_sub]), columns=headers
    )
    #################################################################################################
    if include_motif_information:
        # include_motif_information: 1:12; Odds: raw values; Evens: trend values
        df_motif = get_top_1_motif_numba(
                TimeSeries,
                num_periods_input,
                l=no_points_after_motif,
                include_itself=include_itself,
                compute_trend=(include_motif_information == 2 or include_motif_information == 4 or include_motif_information == 6 or include_motif_information == 8 or include_motif_information == 10 or include_motif_information == 12)
            )
        df_motif_points_after = df_motif[[c for c in df_motif.columns if ("point_after" in c)]]
        if do_normalization:
            df_motif_points_after = df_motif_points_after.sub(df_motif_points_after.mean(axis=1), axis=0).div(df_motif_points_after.std(axis=1, ddof=0), axis=0) # Use Population Standard Deviation (ddof=0)
            df_motif_points_after = df_motif_points_after.replace([np.inf, -np.inf], np.nan)
        Normalized_Data_df = pd.concat([Normalized_Data_df, df_motif_points_after], axis=1)
        # Add motif points before
        if include_motif_information in [3, 4, 5, 6, 9, 10, 11, 12]: # No 7, 8
            df_motif_points_before = df_motif[[c for c in df_motif.columns if ("point_before" in c)]]
            if do_normalization:
                df_motif_points_before = df_motif_points_before.sub(df_motif_points_before.mean(axis=1), axis=0).div(df_motif_points_before.std(axis=1, ddof=0), axis=0) # Use Population Standard Deviation (ddof=0)
                df_motif_points_before = df_motif_points_before.replace([np.inf, -np.inf], np.nan)
            if include_motif_information in [3, 4, 9, 10]:
                # print("df_motif_points_before shape before slicing:", df_motif_points_before.shape)
                df_motif_points_before = df_motif_points_before.iloc[:, -1:]  # only last point
                # print("df_motif_points_before shape after slicing:", df_motif_points_before.shape)
            else:
                pass  # all y points
            Normalized_Data_df = pd.concat([Normalized_Data_df, df_motif_points_before], axis=1)
        if 7 <= include_motif_information <= 12:
            top_1_idx = df_motif[["top_1_motif_idx"]]
            last_point_idx = top_1_idx + num_periods_input -1
            # print("last_point_idx:", last_point_idx)
            # Retrieve time features, handling NaNs in last_point_idx
            idx_values = last_point_idx.values.flatten()
            valid_mask = np.isfinite(idx_values)
            
            # Create container filled with NaNs
            motif_time_features = np.full((len(idx_values), New_sub.shape[1]), np.nan)
            
            if np.any(valid_mask):
                valid_indices = idx_values[valid_mask].astype(int)
                motif_time_features[valid_mask] = New_sub[valid_indices]

            motif_feat_df = pd.DataFrame(motif_time_features, columns=[
                "month_motif", "day_motif", "hour_motif", "minute_motif",
                "day_of_week_motif", "day_of_year_motif", "week_of_year_motif"
            ])
            Normalized_Data_df = pd.concat([Normalized_Data_df, motif_feat_df], axis=1)


        if include_similarity:
            df_motif_dist = df_motif[[c for c in df_motif.columns if ("dist" in c)]]
            # print("Shape of df_motif_dist before normalization:", df_motif_dist.shape)
            df_motif_dist = pd.DataFrame(preprocessing.minmax_scale(df_motif_dist, feature_range=(-0.5, 0.5)), columns=df_motif_dist.columns, index=df_motif_dist.index)
            # print("Shape of df_motif_dist after normalization:", df_motif_dist.shape)
            Normalized_Data_df = pd.concat([Normalized_Data_df, df_motif_dist], axis=1)

 

    #################################################################################################
    # Change 2
    split_index = 11232
    #################################################################################################
    Train = Normalized_Data_df.iloc[0:split_index, :]
    Train = Train.values
    Train = Train.astype("float32")
    Test = Normalized_Data_df.iloc[(split_index - num_periods_input) :, :]
    Test = Test.values
    Test = Test.astype("float32")
    Number_Of_Features = Normalized_Data_df.shape[1]
    ############################################ Windowing ##################################
    end = len(Train)
    start = 0
    next = 0
    x_batches = []
    y_batches = []
    while next + (num_periods_input + num_periods_output) < end:
        next = start + num_periods_input
        x_batches.append(Train[start:next, :])
        y_batches.append(Train[next : next + num_periods_output, 0])
        start = start + 1
    y_batches = np.asarray(y_batches)
    y_batches = y_batches.reshape(-1, num_periods_output, 1)
    x_batches = np.asarray(x_batches)
    x_batches = x_batches.reshape(-1, num_periods_input, Number_Of_Features)
    ############################################ Windowing ##################################
    end_test = len(Test)
    start_test = 0
    next_test = 0
    x_testbatches = []
    y_testbatches = []
    while next_test + (num_periods_input + num_periods_output) < end_test:
        next_test = start_test + num_periods_input
        x_testbatches.append(Test[start_test:next_test, :])
        y_testbatches.append(Test[next_test : next_test + num_periods_output, 0])
        start_test = start_test + 1
    y_testbatches = np.asarray(y_testbatches)
    y_testbatches = y_testbatches.reshape(-1, num_periods_output, 1)
    x_testbatches = np.asarray(x_testbatches)
    x_testbatches = x_testbatches.reshape(-1, num_periods_input, Number_Of_Features)

    if include_covariates and include_motif_information:
        pass
    elif include_covariates and (not include_motif_information):
        pass
    elif (not include_covariates) and include_motif_information:
        selected_cols = np.r_[
            0, len(headers) : Normalized_Data_df.shape[1]
        ]
        x_batches = x_batches[:, :, selected_cols]
        x_testbatches = x_testbatches[:, :, selected_cols]
    else: # (not include_covariates) and (not include_motif_information)
        pass
    return x_batches, y_batches, x_testbatches, y_testbatches

In [5]:
# --- Warmup Run ---
print("Performing Numba warmup (ignoring time)...")
# This compiles the function so the next run is fast
_ = get_top_1_motif_numba(np.random.rand(100), m=10)
print("Warmup complete.\n")

file_name = "pems.npy"
data_path = r"../data/" + file_name
data = np.load(data_path)
data = pd.DataFrame(data)
# data = data[:5]
print("Data shape:", data.shape)

num_periods_input = 9  # input
num_periods_output = 9  # to predict


# ================== Parameters that are going to be changed==================
include_covariates = True  # True/False : whether to include features or not
include_motif_information = 9
no_points_after_motif = 9  # number of points to consider: 1, ceil(m/2), m
do_normalization = False
include_similarity = True
# ================================================
include_itself = True
k_motifs = 1  # 1, 3, 5
# ================================================

xgboost_parameters = {
    "learning_rate": 0.045,
    "n_estimators": 150,
    "max_depth": 8,
    "min_child_weight": 1,
    "gamma": 0.0,
    "subsample": 0.8,
    "colsample_bytree": 0.8,
    "scale_pos_weight": 1,
    "random_state": 42,
    "verbosity": 1,  # 0=Silent, 1=Warning, 2=Info, 3=Debug
    # For GPU acceleration
    "device": "cuda:0",  # Uses the first GPU
    "tree_method": "hist" # Required for modern GPU acceleration
}

# ALL_Test_Data = []
# ALL_Test_Prediction = []

Performing Numba warmup (ignoring time)...
Warmup complete.

Data shape: (228, 12672)


In [6]:
# =================== Processing Time Series ===================
x_batches_Full = []
y_batches_Full = []
X_Test_Full = []
Y_Test_Full = []
for i in range(0, len(data)):
    print("Processing Time Series:", i)
    x_batches = []
    y_batches = []
    X_Test = []
    Y_Test = []
    TimeSeries = data.iloc[i, :]
    x_batches, y_batches, X_Test, Y_Test = New_preprocessing(
        TimeSeries,
        num_periods_input,
        num_periods_output,
        include_itself,
        include_covariates,
        include_motif_information,
        k_motifs,
        no_points_after_motif,
        do_normalization,
        include_similarity
    )
    for element1 in x_batches:
        x_batches_Full.append(element1)

    for element2 in y_batches:
        y_batches_Full.append(element2)

    for element5 in X_Test:
        X_Test_Full.append(element5)

    for element6 in Y_Test:
        Y_Test_Full.append(element6)
## =================== End of Processing Time Series ===================
# Error 136: ValueError: could not broadcast input array from shape (8,) into shape (9,)

Processing Time Series: 0
Processing Time Series: 1
Processing Time Series: 2
Processing Time Series: 3
Processing Time Series: 4
Processing Time Series: 5
Processing Time Series: 6
Processing Time Series: 7
Processing Time Series: 8
Processing Time Series: 9
Processing Time Series: 10
Processing Time Series: 11
Processing Time Series: 12
Processing Time Series: 13
Processing Time Series: 14
Processing Time Series: 15
Processing Time Series: 16
Processing Time Series: 17
Processing Time Series: 18
Processing Time Series: 19
Processing Time Series: 20
Processing Time Series: 21
Processing Time Series: 22
Processing Time Series: 23
Processing Time Series: 24
Processing Time Series: 25
Processing Time Series: 26
Processing Time Series: 27
Processing Time Series: 28
Processing Time Series: 29
Processing Time Series: 30
Processing Time Series: 31
Processing Time Series: 32
Processing Time Series: 33
Processing Time Series: 34
Processing Time Series: 35
Processing Time Series: 36
Processing 

/csproject/kdd/cyuab2/miniconda3/envs/stumpy_gpu/lib/python3.10/site-packages/stumpy/core.py:3467: UserWarning: A large number of values in `P` are smaller than 1e-06.
For a self-join, try setting `ignore_trivial=True`.
  warnings.warn(msg)


Processing Time Series: 142
Processing Time Series: 143
Processing Time Series: 144
Processing Time Series: 145
Processing Time Series: 146
Processing Time Series: 147
Processing Time Series: 148
Processing Time Series: 149
Processing Time Series: 150
Processing Time Series: 151
Processing Time Series: 152
Processing Time Series: 153
Processing Time Series: 154
Processing Time Series: 155
Processing Time Series: 156
Processing Time Series: 157
Processing Time Series: 158
Processing Time Series: 159
Processing Time Series: 160
Processing Time Series: 161
Processing Time Series: 162
Processing Time Series: 163
Processing Time Series: 164
Processing Time Series: 165
Processing Time Series: 166
Processing Time Series: 167
Processing Time Series: 168
Processing Time Series: 169
Processing Time Series: 170
Processing Time Series: 171
Processing Time Series: 172
Processing Time Series: 173
Processing Time Series: 174
Processing Time Series: 175
Processing Time Series: 176
Processing Time Seri

In [7]:
rmse, wape, mae, mape = evaluate_with_xgboost(
    num_periods_output,
    x_batches_Full,
    y_batches_Full,
    X_Test_Full,
    Y_Test_Full,
    xgboost_parameters,
    (include_covariates or (include_motif_information > 0)),
)
print("RMSE: ", rmse)
print("WAPE: ", wape)
print("MAE: ", mae)
print("MAPE: ", mape)

/csproject/kdd/cyuab2/miniconda3/envs/stumpy_gpu/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [12:39:02] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1744329235455/work/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)


RMSE:  5.134441
WAPE:  0.047464203
MAE:  2.7656732
MAPE:  0.067452446


In [8]:
import datetime

print(f"This notebook was last run end-to-end on: {datetime.datetime.now()}\n")
# RMSE:  5.1449504
# WAPE:  0.047548845
# MAE:  2.770605
# MAPE:  0.06760452


# RMSE:  5.126485
# WAPE:  0.04735231
# MAE:  2.7591534
# MAPE:  0.06727818

This notebook was last run end-to-end on: 2026-01-15 12:39:04.429198

